In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class Lazy: # 구간 합 lazy 세그트리
  def __init__(self, size):
    self.size = 1 << (size - 1).bit_length() # size보다 크거나 같은 2의 제곱수(완전트리를 만들기 위함)
    self.tree = [0] * (2 * self.size)
    self.lazy = [0] * (2 * self.size)

  def init(self): # 초기 입력 데이터를 기반으로 세그트리 생성
    for i in reversed(range(self.size)):
      self.tree[i] = self.tree[i*2] + self.tree[i*2+1]

  def propagate(self, i, s, e) :
    if not self.lazy[i] : return #lazy 값이 존재하지 않으면 종료

    self.tree[i] += (e - s) * self.lazy[i] # lazy 값만큼 더하기
    if i < self.size : # 리프 노드가 아니면
      self.lazy[i*2] += self.lazy[i] # 왼쪽 자식의 lazy 값 갱신
      self.lazy[i*2+1] += self.lazy[i] # 오른쪽 자식의 lazy 값 갱신
    self.lazy[i] = 0 # lazy 값 초기화

  def query(self, l, r): # [l, r) 구간의 합을 구한다
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.propagate(i, nl, nr) # propagate를 호출하여 자식들의 lazy 값을 자식들에게 전파
    
    if r <= nl or nr <= l : return 0 # 구간 밖에 있는 경우
    if l <= nl and nr <= r : return self.tree[i] # 구간이 완전히 속하는 경우
    mid = (nl + nr) // 2
    return self._query(l, r, i*2, nl, mid) + self._query(l, r, i*2+1, mid, nr) # 자식 구간에 대해 재귀 호출
  
  def update(self, l, r, x): # [l, r) 구간에 x를 더한다
    self._update(l, r, x, 1, 0, self.size)

  def _update(self, l, r, x, i, nl, nr) :
    self.propagate(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] += x #lazy를 부여한 후 propagate
      self.propagate(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._update(l, r, x, i*2, nl, mid) # 왼쪽 자식 구간 업데이트
    self._update(l, r, x, i*2+1, mid, nr) # 오른쪽 자식 구간 업데이트
    self.tree[i] = self.tree[i*2] + self.tree[i*2+1] # 자식들의 합으로 현재 노드의 합 갱신

def sol() :
  N, M, K = map(int, input().split())
  L = [int(input()) for _ in range(N)]

  lst = Lazy(N + 2)
  for i, v in enumerate(L) :
    lst.tree[lst.size + i] = v
  lst.init()
  
  ans = []
  for _ in range(M + K) :
    q, *l = map(int, input().split())
    if q == 1 :
      s, e, x = l
      lst.update(s-1, e, x)
    elif q == 2 :
      s, e = l
      ans.append(lst.query(s-1, e))
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

### 풀이
- Lazy Propagation을 적용하는 기본 문제.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data, func, default=0, func2=lambda x, y: x*y):
    self.func = func
    self.func2 = func2
    self.default = default
    self.size = 1 << (len(data) - 1).bit_length()
    self.L = [default] * (2 * self.size)
    self.lazy = [default] * (2 * self.size)
    self.L[self.size:self.size + len(data)] = data
    for i in reversed(range(self.size)):
      self.L[i] = self.func(self.L[i*2], self.L[i*2+1])

  def push(self, i, s, e) :
    if self.lazy[i] == self.default : return

    # self.L[i] = self.func(self.L[i], self.func2(e - s, self.lazy[i]))
    if i < self.size :
      self.lazy[i*2] = self.func(self.lazy[i*2], self.lazy[i])
      self.lazy[i*2+1] = self.func(self.lazy[i*2+1], self.lazy[i])
    else :
      self.L[i] = self.func(self.L[i], self.lazy[i])
    self.lazy[i] = self.default

  def query(self, l, r):
    return self._query(l, r, 1, 0, self.size)

  def _query(self, l, r, i, nl, nr) :
    self.push(i, nl, nr)
    
    if r <= nl or nr <= l : return self.default
    if l <= nl and nr <= r : return self.L[i]
    mid = (nl + nr) // 2
    return self.func(self._query(l, r, i*2, nl, mid), self._query(l, r, i*2+1, mid, nr))
  
  def update(self, l, r, x):
    self._update(l, r, x, 1, 0, self.size)

  def _update(self, l, r, x, i, nl, nr) :
    self.push(i, nl, nr)

    if r <= nl or nr <= l : return
    if l <= nl and nr <= r :
      self.lazy[i] = self.func(self.lazy[i], x)
      self.push(i, nl, nr)
      return
  
    mid = (nl + nr) // 2
    self._update(l, r, x, i*2, nl, mid)
    self._update(l, r, x, i*2+1, mid, nr)
    self.L[i] = self.func(self.L[i*2], self.L[i*2+1])



def sol() :
  N, M, K = map(int, input().split())
  L = [int(input()) for _ in range(N)]
  lst = LazySegmentTree(L, lambda x, y: x+y)

  ans = []
  for _ in range(M+K) :
    q, *l = map(int, input().split())
    if q == 1 :
      s, e, x = l
      lst.update(s-1, e, x)
    elif q == 2 :
      s, e = l
      ans.append(lst.query(s-1, e))
  
  sys.stdout.write('\n'.join(map(str, ans)))

sol()

- 필요 이상의 Adaptability?